<a href="https://colab.research.google.com/github/MaJu502/proyectoDL/blob/main/ProyectoDL2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python

In [ ]:
!pip install mtcnn
#https://medium.com/lcc-unison/aplicaci%C3%B3n-de-reconocimiento-facial-en-tiempo-real-para-identificar-a-alumnos-de-la-licenciatura-en-2e2b53870995

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.8 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
from mtcnn import MTCNN
from torchvision.transforms import functional as F

In [ ]:
import os
import cv2

# Carpeta que contiene los videos
video_folder = 'validation_videos/'

# Carpeta de salida para guardar los frames
output_folder = 'validation_frames/'

# Crea la carpeta de salida si no existe
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Lista de archivos en la carpeta de videos
video_files = os.listdir(video_folder)

for video_file in video_files:
    if video_file.endswith('.mp4') or video_file.endswith('.avi'):
        video_path = os.path.join(video_folder, video_file)

        # Abre el video
        cap = cv2.VideoCapture(video_path)

        # Verifica si el video se abrió correctamente
        if not cap.isOpened():
            print(f"No se pudo abrir el video: {video_path}")
            continue

        frame_count = 0
        frame_interval = 5  # Captura un frame cada 5 frames

        # Lee y guarda los frames como imágenes
        while True:
            ret, frame = cap.read()

            if not ret:
                break

            if frame_count % frame_interval == 0:
                # Guarda el frame como una imagen
                frame_filename = os.path.join(output_folder, f'{os.path.splitext(video_file)[0]}_frame_{frame_count:04d}.png')
                cv2.imwrite(frame_filename, frame)

            frame_count += 1

        # Cierra el video
        cap.release()

        print(f'Se han guardado {frame_count // frame_interval} frames del video {video_file} como imágenes.')

print('Proceso completo.')

Se han guardado 72 frames del video face_recognition_1.mp4 como imágenes.
Se han guardado 41 frames del video face_recognition_2.mp4 como imágenes.
Proceso completo.


In [ ]:
# Model YOLO
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
# Model MTCNN
face_detector = MTCNN()

# Carpeta de salida para guardar resultados
output_folder = 'persons/'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Carpeta que contiene los frames de validación
folder = 'validation_frames/'  # Cambiar el nombre de la carpeta de frames de validación
files = os.listdir(folder)

# Listas para métricas
true_positives = 0
false_positives = 0
false_negatives = 0

# Evaluar cada elemento dentro de los frames de validación
for img_name in files:
    img_path = os.path.join("validation_frames", img_name)  # Ruta de la carpeta de validación
    img = Image.open(img_path)
    results = model(img)

    # Obtener resultados de bounding boxes
    bboxes = results.pred[0]

    # Flag para seguimiento de detecciones
    detection_flag = False

    for i, bbox in enumerate(bboxes):
        # Ver si la clase es de tipo person
        if bbox[-1] == 0:
            x1, y1, x2, y2 = bbox[0:4].int().tolist()

            # Recortar imagen para obtener únicamente la caja
            cropped_img = img.crop((x1, y1, x2, y2))

            # Recortar caja para obtener únicamente las caras
            image_np = np.array(cropped_img)
            face_results = face_detector.detect_faces(image_np)

            if len(face_results) > 0:
                detection_flag = True
                face_bbox = face_results[0]['box']
                x, y, w, h = face_bbox
                face_image = cropped_img.crop((x, y, x + w, y + h))

                # Guardar imagen en carpeta persona
                output_path = os.path.join(output_folder, f"person_{os.path.splitext(img_name)[0]}_{i}.jpg")
                face_image.save(output_path)

    if detection_flag:
        true_positives += 1
    else:
        false_negatives += 1

# Calcular falsos positivos
false_positives = len(files) - true_positives

# Calcular métricas
precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)
f1_score = 2 * (precision * recall) / (precision + recall)

print(f"True Positives: {true_positives}")
print(f"False Positives: {false_positives}")
print(f"False Negatives: {false_negatives}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")

In [ ]:
pip install joblib

In [ ]:
import joblib

# Guardar el modelo YOLO
yolo_model_path = 'yolo_model.pkl'
joblib.dump(model, yolo_model_path)

# Guardar el modelo MTCNN (asegúrate de haber importado MTCNN correctamente)
mtcnn_model_path = 'mtcnn_model.pkl'
joblib.dump(face_detector, mtcnn_model_path)


['mtcnn_model.pkl']

In [ ]:
# Cargar el modelo YOLO
loaded_yolo_model = joblib.load(yolo_model_path)

# Cargar el modelo MTCNN
loaded_mtcnn_model = joblib.load(mtcnn_model_path)
